# Final Data Processing

In [61]:
import numpy as np
import pandas as pd

import warnings
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.3f}'.format
warnings.filterwarnings('ignore')

In [62]:
train = pd.read_csv("../data/train.csv")
train['date'] = pd.to_datetime(train['date'])

In [63]:
family_rank = train.groupby("family").sales.mean().sort_values(ascending = False).reset_index()

In [64]:
# 전체 sales의 99% 기준 계산
total_sales = family_rank['sales'].sum()
cumulative_sales = family_rank['sales'].cumsum()
threshold = total_sales * 0.99

# 95%를 차지하는 품목 식별
family_rank['cumulative_sales'] = cumulative_sales
top_99 = family_rank[family_rank['cumulative_sales'] <= threshold]
top_99_families = top_99['family'].unique()

# 나머지 품목 합산하여 OTHERS로 처리
others = family_rank[family_rank['cumulative_sales'] > threshold]
others_sum = others['sales'].sum()

# 결과 데이터프레임 생성
result = top_99[['family', 'sales']].copy()
others_row = pd.DataFrame([{'family': 'OTHERS', 'sales': others_sum}])
result = pd.concat([result, others_row], ignore_index=True)

result

,family,sales
0,GROCERY I,3776.972
1,BEVERAGES,2385.793
2,PRODUCE,1349.352
3,CLEANING,1072.417
4,DAIRY,709.155
5,BREAD/BAKERY,463.336
6,POULTRY,350.532
7,MEATS,341.850
8,PERSONAL CARE,270.433
9,DELI,265.135


In [65]:
# family가 top_95 목록에 없으면 'OTHERS'로 변경
train['family'] = train['family'].apply(lambda x: x if x in top_99_families else 'OTHERS')

In [66]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB


In [67]:
# 'date'와 'store_nbr'로 그룹화하고, 'family'가 'OTHERS'인 경우를 처리
def process_group(group):
    if 'OTHERS' in group['family'].values:
        # 'OTHERS'에 해당하는 'sales'와 'onpromotion' 값 합산
        others_sales_sum = group[group['family'] == 'OTHERS']['sales'].sum()
        others_onpromotion_sum = group[group['family'] == 'OTHERS']['onpromotion'].sum()
        # 'OTHERS' 중 첫 번째 항목만 남기고 나머지는 삭제
        first_others_idx = group[group['family'] == 'OTHERS'].index[0]
        group = group.drop(group[group['family'] == 'OTHERS'].index[1:])
        # 남은 'OTHERS'의 'sales'와 'onpromotion' 값을 합산된 값으로 업데이트
        group.loc[first_others_idx, 'sales'] = others_sales_sum
        group.loc[first_others_idx, 'onpromotion'] = others_onpromotion_sum
    return group

# 그룹별로 처리
train_pre = train.groupby(['date', 'store_nbr']).apply(process_group).reset_index(drop=True)
train_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454976 entries, 0 to 1454975
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   id           1454976 non-null  int64         
 1   date         1454976 non-null  datetime64[ns]
 2   store_nbr    1454976 non-null  int64         
 3   family       1454976 non-null  object        
 4   sales        1454976 non-null  float64       
 5   onpromotion  1454976 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 66.6+ MB


In [68]:
train_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454976 entries, 0 to 1454975
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   id           1454976 non-null  int64         
 1   date         1454976 non-null  datetime64[ns]
 2   store_nbr    1454976 non-null  int64         
 3   family       1454976 non-null  object        
 4   sales        1454976 non-null  float64       
 5   onpromotion  1454976 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 66.6+ MB


##### transaction 처리

In [69]:
transactions = pd.read_csv("../data//transactions.csv")
transactions['date'] = pd.to_datetime(transactions['date'])
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.9 MB


In [70]:
# 데이터프레임 병합
train_transactions = pd.merge(train_pre, transactions, on=['date', 'store_nbr'], how='left')

# 결과 출력
train_transactions

,id,date,store_nbr,family,sales,onpromotion,transactions
0,0,2013-01-01,1,OTHERS,0.000,0,NaN
1,3,2013-01-01,1,BEVERAGES,0.000,0,NaN
2,5,2013-01-01,1,BREAD/BAKERY,0.000,0,NaN
3,7,2013-01-01,1,CLEANING,0.000,0,NaN
4,8,2013-01-01,1,DAIRY,0.000,0,NaN
...,...,...,...,...,...,...,...
1454971,3000747,2017-08-15,54,MEATS,57.842,0,802.000
1454972,3000748,2017-08-15,54,PERSONAL CARE,169.000,5,802.000
1454973,3000751,2017-08-15,54,POULTRY,59.619,0,802.000
1454974,3000752,2017-08-15,54,PREPARED FOODS,94.000,0,802.000


In [71]:
missing_sales = train_transactions['sales'].isnull().sum()
sales_zero_count = train_transactions[train_transactions['sales'] == 0.0].shape[0]

missing_transactions = train_transactions['transactions'].isnull().sum()
transactions_zero_count = train_transactions[train_transactions['transactions'] == 0.0].shape[0]

print(f"Missing sales : {missing_sales}, Number of rows with sales value of 0: {sales_zero_count}")
print(f"Missing transactions : {missing_transactions}, Number of rows with sales value of 0: {transactions_zero_count}")


Missing sales : 0, Number of rows with sales value of 0: 174603
Missing transactions : 119168, Number of rows with sales value of 0: 0


In [72]:
# sales 값이 0인 행 삭제
train_transactions = train_transactions[train_transactions['sales'] != 0.0]

# transactions의 null 값을 0으로 설정
train_transactions['transactions'].fillna(0, inplace=True)

# 결과 확인
missing_sales = train_transactions['sales'].isnull().sum()
sales_zero_count = train_transactions[train_transactions['sales'] == 0.0].shape[0]

missing_transactions = train_transactions['transactions'].isnull().sum()
transactions_zero_count = train_transactions[train_transactions['transactions'] == 0.0].shape[0]

print(f"Missing sales : {missing_sales}, Number of rows with sales value of 0: {sales_zero_count}")
print(f"Missing transactions : {missing_transactions}, Number of rows with transactions value of 0: {transactions_zero_count}")

Missing sales : 0, Number of rows with sales value of 0: 0
Missing transactions : 0, Number of rows with transactions value of 0: 1826


In [73]:
# 인덱스 재설정
train_transactions.reset_index(drop=True, inplace=True)

In [74]:
train_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1280373 entries, 0 to 1280372
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   id            1280373 non-null  int64         
 1   date          1280373 non-null  datetime64[ns]
 2   store_nbr     1280373 non-null  int64         
 3   family        1280373 non-null  object        
 4   sales         1280373 non-null  float64       
 5   onpromotion   1280373 non-null  int64         
 6   transactions  1280373 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 68.4+ MB


300만개 데이터를 128만개로 줄임

##### holidays_events 처리

In [75]:
holidays_events = pd.read_csv("../data//holidays_events.csv")
holidays_events['date'] = pd.to_datetime(transactions['date'])
holidays_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         350 non-null    datetime64[ns]
 1   type         350 non-null    object        
 2   locale       350 non-null    object        
 3   locale_name  350 non-null    object        
 4   description  350 non-null    object        
 5   transferred  350 non-null    bool          
dtypes: bool(1), datetime64[ns](1), object(4)
memory usage: 14.1+ KB


preprocessing 결과 locale과 type에 다른 변수에 따라 영향을 받지 않으므로 단순히 holday 가 있는 날은 0, 없는 날을 1로 세팅

In [76]:
train_transactions['is_holiday'] = train_transactions['date'].isin(holidays_events['date']).astype(int)

In [77]:
train_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1280373 entries, 0 to 1280372
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   id            1280373 non-null  int64         
 1   date          1280373 non-null  datetime64[ns]
 2   store_nbr     1280373 non-null  int64         
 3   family        1280373 non-null  object        
 4   sales         1280373 non-null  float64       
 5   onpromotion   1280373 non-null  int64         
 6   transactions  1280373 non-null  float64       
 7   is_holiday    1280373 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 78.1+ MB


##### store 데이터 처리

In [78]:
stores = pd.read_csv("../data//stores.csv")
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


In [79]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [80]:
train_stores = pd.merge(train_transactions, stores, on='store_nbr', how='left')

In [81]:
train_stores

,id,date,store_nbr,family,sales,onpromotion,transactions,is_holiday,city,state,type,cluster
0,561,2013-01-01,25,OTHERS,24.000,0,770.000,1,Salinas,Santa Elena,D,1
1,564,2013-01-01,25,BEVERAGES,810.000,0,770.000,1,Salinas,Santa Elena,D,1
2,566,2013-01-01,25,BREAD/BAKERY,180.589,0,770.000,1,Salinas,Santa Elena,D,1
3,568,2013-01-01,25,CLEANING,186.000,0,770.000,1,Salinas,Santa Elena,D,1
4,569,2013-01-01,25,DAIRY,143.000,0,770.000,1,Salinas,Santa Elena,D,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1280368,3000747,2017-08-15,54,MEATS,57.842,0,802.000,0,El Carmen,Manabi,C,3
1280369,3000748,2017-08-15,54,PERSONAL CARE,169.000,5,802.000,0,El Carmen,Manabi,C,3
1280370,3000751,2017-08-15,54,POULTRY,59.619,0,802.000,0,El Carmen,Manabi,C,3
1280371,3000752,2017-08-15,54,PREPARED FOODS,94.000,0,802.000,0,El Carmen,Manabi,C,3


#### oil data

In [82]:
oil = pd.read_csv("../data/oil.csv")
oil['date'] = pd.to_datetime(oil['date'])

In [83]:
oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1218 non-null   datetime64[ns]
 1   dcoilwtico  1175 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 19.2 KB


In [84]:
# Resample
oil = oil.set_index("date").dcoilwtico.resample("D").sum().reset_index()
# Interpolate
oil["dcoilwtico"] = np.where(oil["dcoilwtico"] == 0, np.nan, oil["dcoilwtico"])
oil["dcoilwtico_interpolated"] =oil.dcoilwtico.interpolate()
oil.drop(columns=['dcoilwtico'], inplace=True)
oil['dcoilwtico_interpolated'] = oil['dcoilwtico_interpolated'].fillna(method='bfill')
oil.head()



,date,dcoilwtico_interpolated
0,2013-01-01,93.140
1,2013-01-02,93.140
2,2013-01-03,92.970
3,2013-01-04,93.120
4,2013-01-05,93.147


In [85]:
train_all = pd.merge(train_stores, oil, on='date', how='left')
train_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1280373 entries, 0 to 1280372
Data columns (total 13 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   id                       1280373 non-null  int64         
 1   date                     1280373 non-null  datetime64[ns]
 2   store_nbr                1280373 non-null  int64         
 3   family                   1280373 non-null  object        
 4   sales                    1280373 non-null  float64       
 5   onpromotion              1280373 non-null  int64         
 6   transactions             1280373 non-null  float64       
 7   is_holiday               1280373 non-null  int64         
 8   city                     1280373 non-null  object        
 9   state                    1280373 non-null  object        
 10  type                     1280373 non-null  object        
 11  cluster                  1280373 non-null  int64         
 12  

In [86]:
train_all['weekday'] = train_all['date'].dt.dayofweek + 1 # 1: 월요일, 7:일요일

train_all['year'] = train_all['date'].dt.year
train_all['month'] = train_all['date'].dt.month
train_all['day'] = train_all['date'].dt.day

In [87]:
# 이동 평균 및 이동 표준 편차 계산
train_all['Rolling_Std_Sales_3d'] = train_all['sales'].rolling(window=3, min_periods=1).std()
train_all['Rolling_Mean_Sales_7d'] = train_all['sales'].rolling(window=7, min_periods=1).mean()
train_all['Rolling_Std_Sales_7d'] = train_all['sales'].rolling(window=7, min_periods=1).std()
train_all['Rolling_Mean_Sales_14d'] = train_all['sales'].rolling(window=14, min_periods=1).mean()
train_all['Rolling_Std_Sales_14d'] = train_all['sales'].rolling(window=14, min_periods=1).std()

# 결측치를 다음 값으로 채우기
train_all['Rolling_Std_Sales_3d'].fillna(method='bfill', inplace=True)
train_all['Rolling_Mean_Sales_7d'].fillna(method='bfill', inplace=True)
train_all['Rolling_Std_Sales_7d'].fillna(method='bfill', inplace=True)
train_all['Rolling_Mean_Sales_14d'].fillna(method='bfill', inplace=True)
train_all['Rolling_Std_Sales_14d'].fillna(method='bfill', inplace=True)

In [88]:
train_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1280373 entries, 0 to 1280372
Data columns (total 22 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   id                       1280373 non-null  int64         
 1   date                     1280373 non-null  datetime64[ns]
 2   store_nbr                1280373 non-null  int64         
 3   family                   1280373 non-null  object        
 4   sales                    1280373 non-null  float64       
 5   onpromotion              1280373 non-null  int64         
 6   transactions             1280373 non-null  float64       
 7   is_holiday               1280373 non-null  int64         
 8   city                     1280373 non-null  object        
 9   state                    1280373 non-null  object        
 10  type                     1280373 non-null  object        
 11  cluster                  1280373 non-null  int64         
 12  

In [89]:
train_all.to_csv('train_all.csv', index=False)